In [8]:
import pickle

fd = open('trajectories.pickle','rb')
D = pickle.load(fd)

In [11]:
len(D)

1000

In [12]:
len(D[0])

500

In [13]:
len(D[0][0])

4

In [26]:
D[0][0:9]

[((15, 0), 0, -0.5058824, (14, 0)),
 ((14, 0), 6, -0.5058824, (14, 0)),
 ((14, 0), 5, -0.5058824, (15, 1)),
 ((15, 1), 3, -0.5058824, (16, 0)),
 ((16, 0), 3, -0.5058824, (17, 0)),
 ((17, 0), 6, -0.5058824, (17, 0)),
 ((17, 0), 0, -0.5058824, (16, 0)),
 ((16, 0), 5, -0.5058824, (17, 0)),
 ((17, 0), 1, -0.5058824, (16, 0))]

In [27]:
D[0][100:109]

[((21, 7), 6, -0.2, (21, 6)),
 ((21, 6), 1, -0.5058824, (20, 7)),
 ((20, 7), 6, -0.41960785, (20, 6)),
 ((20, 6), 0, -0.5058824, (19, 5)),
 ((19, 5), 6, -0.5882353, (19, 4)),
 ((19, 4), 6, -0.6509804, (19, 3)),
 ((19, 3), 6, -0.627451, (19, 2)),
 ((19, 2), 5, -0.5411765, (20, 3)),
 ((20, 3), 3, -0.5058824, (21, 2))]